<a href="https://colab.research.google.com/github/rrupam/Bias-in-toxicity-classification/blob/master/bias_toxic_comment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm, trange
from sklearn.metrics import roc_auc_score
import pickle
import gc

In [0]:
os.listdir()

['.config', 'drive', 'sample_data']

In [0]:
BERT_MODEL = 'bert-base-uncased'
CASED = 'uncased' in BERT_MODEL
INPUT = '../input/jigsaw-bert-preprocessed-input/'
TEXT_COL = 'comment_text'
MAXLEN = 220

In [0]:
train = pd.read_csv('drive/My Drive/Colab Notebooks/train.csv', nrows = 100000)

In [0]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.0,4,47


In [0]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 45 columns):
id                                     100000 non-null int64
target                                 100000 non-null float64
comment_text                           100000 non-null object
severe_toxicity                        100000 non-null float64
obscene                                100000 non-null float64
identity_attack                        100000 non-null float64
insult                                 100000 non-null float64
threat                                 100000 non-null float64
asian                                  19625 non-null float64
atheist                                19625 non-null float64
bisexual                               19625 non-null float64
black                                  19625 non-null float64
buddhist                               19625 non-null float64
christian                              19625 non-null float64
female           

In [0]:
test = pd.read_csv('drive/My Drive/Colab Notebooks/test.csv')

In [0]:
test.head()

,id,comment_text
0,7000000,Jeff Sessions is another one of Trump's Orwell...
1,7000001,I actually inspected the infrastructure on Gra...
2,7000002,No it won't . That's just wishful thinking on ...
3,7000003,Instead of wringing our hands and nibbling the...
4,7000004,how many of you commenters have garbage piled ...


In [0]:
pd.set_option('display.max_colwidth', -1)
print(train[11:12]['comment_text'])

11    This is a great story. Man. I wonder if the person who yelled "shut the fuck up!" at him ever heard it.
Name: comment_text, dtype: object


In [0]:
train = train.fillna(0)

In [0]:
train[train['target']>0.5].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5134 entries, 4 to 99973
Data columns (total 45 columns):
id                                     5134 non-null int64
target                                 5134 non-null float64
comment_text                           5134 non-null object
severe_toxicity                        5134 non-null float64
obscene                                5134 non-null float64
identity_attack                        5134 non-null float64
insult                                 5134 non-null float64
threat                                 5134 non-null float64
asian                                  5134 non-null float64
atheist                                5134 non-null float64
bisexual                               5134 non-null float64
black                                  5134 non-null float64
buddhist                               5134 non-null float64
christian                              5134 non-null float64
female                                 51

In [0]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:41.987077+00,2,0.0,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,"Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:42.870083+00,2,0.0,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos to you for taking it on. Very impressive!,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:45.222647+00,2,0.0,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on my site? When will you be releasing it?,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:47.601894+00,2,0.0,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,0.0,2006,rejected,0,0,0,1,0,0.0,4,47


In [0]:
import sys
package_dir = "../input/ppbert/pytorch-pretrained-bert/pytorch-pretrained-BERT"
sys.path.append(package_dir)

In [0]:
!pip install pytorch-pretrained-bert

     |████████████████████████████████| 133kB 54.0MB/s 
     |████████████████████████████████| 655kB 44.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/35/e4/80/abf3b33ba89cf65cd262af8a22a5a999cc28fbfabea6b38473
Successfully built regex


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import torch.utils.data
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import warnings
#import pytorch_pretrained_bert
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam
from pytorch_pretrained_bert import BertConfig
from pytorch_pretrained_bert.modeling import BertModel, BertForMaskedLM


warnings.filterwarnings(action='once')
device = torch.device('cuda')

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 492025.57B/s]


In [0]:
IDENTITY_COLUMNS = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness'
]
AUX_COLUMNS = [ 'severe_toxicity', 'obscene', 'identity_attack', 'insult', 'threat']
TEXT_COLUMN = 'comment_text'
TARGET_COLUMN = 'target'

In [0]:
x_train = pd.DataFrame(train[TEXT_COLUMN])
y_train = train[TARGET_COLUMN]
y_aux_train = train[AUX_COLUMNS].values
y_id_train = train[IDENTITY_COLUMNS].values
x_test = pd.DataFrame(test[TEXT_COLUMN].astype(str))

for column in IDENTITY_COLUMNS + [TARGET_COLUMN]:
    train[column] = np.where(train[column] >= 0.5, True, False)

In [79]:
y_aux_train[0]

array([0., 0., 0., 0., 0.])

We tokenize the text using the lambda function

In [0]:

x_t = x_train.apply(lambda x: tokenizer.tokenize(x['comment_text']), axis=1)

In [0]:
x_test = x_test.apply(lambda x: tokenizer.tokenize(x['comment_text']), axis=1)

We add 'CLS' to the beginning of a comment and 'SEP' to the end of it as BertforSequenceClasssification requires this.

In [0]:
x_t = ["[CLS]"] + x_t + ["[SEP]"]

In [36]:
print(x_t[100])

['[CLS]', 'loving', 'this', 'collection', '.', 'can', '##t', 'wait', 'till', 'season', '2', 'is', 'released', '.', 'should', 'be', 'any', 'day', 'now', 'according', 'to', 'http', ':', '/', '/', 'ye', '##ez', '##y', '-', 'season', '##2', '.', 'com', '/', '[SEP]']


In [37]:
x_test =["[CLS]"] + x_test + ["[SEP]"]
print(x_test[100])

['[CLS]', 'did', 'you', 'even', 'read', 'the', 'editorial', '?', '"', 'the', 'best', 'course', 'is', 'for', 'the', 'democratic', 'world', 'to', 'continue', 'to', 'demand', 'the', 'return', 'of', 'the', 'country', '’', 's', 'legislature', ',', 'and', 'the', 'end', 'of', 'the', 'sham', 'constituent', 'assembly', 'that', 'us', '##ur', '##ped', 'it', '.', 'imp', '##art', '##ial', 'outsiders', ',', 'working', 'with', 'both', 'the', 'opposition', 'and', 'mr', '.', 'mad', '##uro', ',', 'could', 'help', 'negotiate', 'a', 'schedule', 'for', 'the', 'return', 'of', 'legitimate', 'elections', '.', '"', 'how', 'exactly', 'do', 'you', 'interpret', 'that', 'as', 'a', 'call', 'for', 'a', 'military', 'invasion', '?', '[SEP]']


Convert the tokens to ids for the training set

In [0]:
x_train = pd.DataFrame(x_t)
x_train = x_train.apply(lambda x: tokenizer.convert_tokens_to_ids(x[0]), axis=1)

Convert the tokens into ids for the test set

In [39]:
x_test = pd.DataFrame(x_test)
x_test = x_test.apply(lambda x: tokenizer.convert_tokens_to_ids(x[0]), axis=1)

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (547 > 512). Running this sequence through BERT will result in indexing errors


Convert the tokens into ids using BERT embeddings 

Look at a sample comment converted to id.

In [0]:
print(x_test[100])

[2106, 2017, 2130, 3191, 1996, 8368, 1029, 1000, 1996, 2190, 2607, 2003, 2005, 1996, 3537, 2088, 2000, 3613, 2000, 5157, 1996, 2709, 1997, 1996, 2406, 1521, 1055, 6372, 1010, 1998, 1996, 2203, 1997, 1996, 25850, 13794, 3320, 2008, 2149, 3126, 5669, 2009, 1012, 17727, 8445, 4818, 22361, 1010, 2551, 2007, 2119, 1996, 4559, 1998, 2720, 1012, 5506, 10976, 1010, 2071, 2393, 13676, 1037, 6134, 2005, 1996, 2709, 1997, 11476, 3864, 1012, 1000, 2129, 3599, 2079, 2017, 17841, 2008, 2004, 1037, 2655, 2005, 1037, 2510, 5274, 1029]


In [42]:
from keras.preprocessing import sequence
x_train = sequence.pad_sequences(x_train, maxlen=MAXLEN)
x_test = sequence.pad_sequences(x_test, maxlen=MAXLEN)
print(x_train[0])

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:5506: ResourceWarning: unclosed file <_io.TextIOWrapper name='/root/.keras/keras.json' mode='r' encoding='UTF-8'>
  _config = json.load(open(_config_path))
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0  101 2023 2003 2061 4658 1012 2009
 1005 

In [0]:
x_train_tensor = torch.tensor(x_train)


In [82]:
print(type(y_aux_train))
print(y_aux_train[0:1])
y_aux_train = pd.DataFrame([y_aux_train])
y_aux_train_tensor = torch.tensor(y_aux_train)
print(y_aux_train[0:1])

<class 'pandas.core.frame.DataFrame'>
     0    1    2    3    4
0  0.0  0.0  0.0  0.0  0.0


KeyError: ignored

In [0]:
from pytorch_pretrained_bert import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = len(y_aux_train))


In [0]:
batch_size = 16

train_lists = [x_train_tensor, y_train]
test_lists = [X_test, y_test]

training_dataset = text_dataset(x_y_list = train_lists )

test_dataset = text_dataset(x_y_list = test_lists )

dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    print('starting')
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            
            sentiment_corrects = 0
            
            
            # Iterate over data.
            for inputs, sentiment in dataloaders_dict[phase]:
                #inputs = inputs
                #print(len(inputs),type(inputs),inputs)
                #inputs = torch.from_numpy(np.array(inputs)).to(device) 
                inputs = inputs.to(device) 

                sentiment = sentiment.to(device)
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    #print(inputs)
                    outputs = model(inputs)

                    outputs = F.softmax(outputs,dim=1)
                    
                    loss = criterion(outputs, torch.max(sentiment.float(), 1)[1])
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)

                
                sentiment_corrects += torch.sum(torch.max(outputs, 1)[1] == torch.max(sentiment, 1)[1])

                
            epoch_loss = running_loss / dataset_sizes[phase]

            
            sentiment_acc = sentiment_corrects.double() / dataset_sizes[phase]

            print('{} total loss: {:.4f} '.format(phase,epoch_loss ))
            print('{} sentiment_acc: {:.4f}'.format(
                phase, sentiment_acc))

            if phase == 'val' and epoch_loss < best_loss:
                print('saving with loss of {}'.format(epoch_loss),
                      'improved over previous {}'.format(best_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'bert_model_test.pth')


        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(float(best_loss)))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [53]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediat

In [0]:
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler
import time
import copy


lrlast = .001
lrmain = .00001
optim1 = optim.Adam(
    [
        {"params":model.bert.parameters(),"lr": lrmain},
        {"params":model.classifier.parameters(), "lr": lrlast},
       
   ])

#optim1 = optim.Adam(model.parameters(), lr=0.001)#,momentum=.9)
# Observe that all parameters are being optimized
optimizer_ft = optim1
criterion = nn.CrossEntropyLoss()

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

In [62]:
model_ft1 = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=10)


starting
Epoch 0/9
----------


NameError: ignored